In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



In [14]:
# 使用FashionMNIST 数据集
# 每个Dataset包含两个参数, transform和target_transform, 来对样本和对应的标签进行处理

# 下载公开的训练集
training_data = datasets.FashionMNIST(
    root = './data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# 下载测试集
test_data = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=ToTensor(),
)


In [15]:
# 将Dataset作为DataLoader的参数, 对数据集进行包裹,
# 可以支持自动分批, 采样, 混洗和多进程加载

batch_size = 64

# 创建数据加载器
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape}')
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [16]:
# 创建模型
"""
创建一个继承自nn.Module的类, 在__init__函数中定义
网络的层, 在forward函数中指定数据如何通过网络
"""
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("cpu")

class NeuralNetWork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetWork().to(device)
print(model)




cpu
NeuralNetWork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
# 优化模型参数
# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss:{loss:>7f}, [{current:>5d} / {size:>5f}]')
        

In [20]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error:\n Accuracy:{(100 * correct):>0.1f}%, Avg loss:{test_loss:>0.8f}\n")
        

In [22]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n{'-' * 20}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
--------------------
loss:1.186712, [    0 / 60000.000000]
loss:1.192025, [ 6400 / 60000.000000]
loss:1.017774, [12800 / 60000.000000]
loss:1.135395, [19200 / 60000.000000]
loss:1.005876, [25600 / 60000.000000]
loss:1.056735, [32000 / 60000.000000]
loss:1.062182, [38400 / 60000.000000]
loss:1.016750, [44800 / 60000.000000]
loss:1.068795, [51200 / 60000.000000]
loss:0.983841, [57600 / 60000.000000]
Test Error:
 Accuracy:65.5%, Avg loss:1.00472479

Epoch 2
--------------------
loss:1.062752, [    0 / 60000.000000]
loss:1.092763, [ 6400 / 60000.000000]
loss:0.899624, [12800 / 60000.000000]
loss:1.040598, [19200 / 60000.000000]
loss:0.913779, [25600 / 60000.000000]
loss:0.961334, [32000 / 60000.000000]
loss:0.984064, [38400 / 60000.000000]
loss:0.937883, [44800 / 60000.000000]
loss:0.985816, [51200 / 60000.000000]
loss:0.917800, [57600 / 60000.000000]
Test Error:
 Accuracy:66.8%, Avg loss:0.92954776

Epoch 3
--------------------
loss:0.971182, [    0 / 60000.000000]
loss:1.023809, 

KeyboardInterrupt: 